### Lab2 (a) Model preperation

In [12]:
from resnet20 import ResNetCIFAR
from train_util import train, finetune, test
import torch
import numpy as np

import time

import torchvision.transforms as transforms
import torchvision
import torch.nn as nn
import torch.optim as optim

from FP_layers import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
net = ResNetCIFAR(num_layers=20, Nbits=None)
net = net.to(device)

In [6]:
# Load the best weight paramters
net.load_state_dict(torch.load("pretrained_model.pt"))
test(net)

Files already downloaded and verified
Test Loss=0.3231, Test accuracy=0.9151


### Lab2 (b) Prune by percentage

In [2]:
def prune_by_percentage(layer, q=80.0):
    """
    Pruning the weight paramters by threshold.
    :param q: pruning percentile. 'q' percent of the least 
    significant weight parameters will be pruned.
    """
    # Convert the weight of "layer" to numpy array
    weight = layer.weight.data.cpu().numpy()
    # Compute the q-th percentile of the abs of the converted array
    threshold = np.percentile(np.abs(weight), q)
    # Generate a binary mask same shape as weight to decide which element to prune
    mask = np.abs(weight) > threshold
    # Convert mask to torch tensor and put on GPU
    mask = torch.from_numpy(mask).float().to(device)
    # Multiply the weight by mask to perform pruning
    layer.weight.data.mul_(mask)
    pass

In [28]:
# try q = 0.4, 0.6, 0.8
q_lst = [40,60,80]
net.load_state_dict(torch.load("pretrained_model.pt"))

for q in q_lst:
    print("q:",str(q))
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # change q value
            prune_by_percentage(layer, q)
            # Optional: Check the sparsity you achieve in each layer
            print(name, layer.weight.data.cpu().numpy().size - np.count_nonzero(layer.weight.data.cpu().numpy()))
            # Convert the weight of "layer" to numpy array
            np_weight = layer.weight.data.cpu().numpy()
            # Count number of zeros
            zeros = np.count_nonzero(np_weight == 0) 
            # Count number of parameters
            total = np_weight.size
            # Print sparsity
            print('Sparsity of '+name+': '+str(zeros/total))

    test(net)

q: 40
head_conv.0.conv 173
Sparsity of head_conv.0.conv: 0.40046296296296297
body_op.0.conv1.0.conv 922
Sparsity of body_op.0.conv1.0.conv: 0.4001736111111111
body_op.0.conv2.0.conv 922
Sparsity of body_op.0.conv2.0.conv: 0.4001736111111111
body_op.1.conv1.0.conv 922
Sparsity of body_op.1.conv1.0.conv: 0.4001736111111111
body_op.1.conv2.0.conv 922
Sparsity of body_op.1.conv2.0.conv: 0.4001736111111111
body_op.2.conv1.0.conv 922
Sparsity of body_op.2.conv1.0.conv: 0.4001736111111111
body_op.2.conv2.0.conv 922
Sparsity of body_op.2.conv2.0.conv: 0.4001736111111111
body_op.3.conv1.0.conv 1843
Sparsity of body_op.3.conv1.0.conv: 0.3999565972222222
body_op.3.conv2.0.conv 3687
Sparsity of body_op.3.conv2.0.conv: 0.4000651041666667
body_op.4.conv1.0.conv 3687
Sparsity of body_op.4.conv1.0.conv: 0.4000651041666667
body_op.4.conv2.0.conv 3687
Sparsity of body_op.4.conv2.0.conv: 0.4000651041666667
body_op.5.conv1.0.conv 3687
Sparsity of body_op.5.conv1.0.conv: 0.4000651041666667
body_op.5.conv2.

### Lab2 (c) Finetune pruned model

In [3]:
def finetune_after_prune(net, trainloader, criterion, optimizer, prune=True):
    """
    Finetune the pruned model for a single epoch
    Make sure pruned weights are kept as zero
    """
    # Build a dictionary for the nonzero weights
    weight_mask = {}
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Your code here: generate a mask in GPU torch tensor to have 1 for nonzero element and 0 for zero element 
            weight_mask[name] = torch.from_numpy(np.abs(layer.weight.data.cpu().numpy()) > 0).float().to(device)
    
    global_steps = 0
    train_loss = 0
    correct = 0
    total = 0
    start = time.time()
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        if prune:
            for name,layer in net.named_modules():
                if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
                    # Your code here: Use weight_mask to make sure zero elements remains zero
                    layer.weight.data = layer.weight.data.mul_(weight_mask[name])
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        global_steps += 1

        if global_steps % 50 == 0:
            end = time.time()
            batch_size = 256
            num_examples_per_second = 50 * batch_size / (end - start)
            print("[Step=%d]\tLoss=%.4f\tacc=%.4f\t%.1f examples/second"
                 % (global_steps, train_loss / (batch_idx + 1), (correct / total), num_examples_per_second))
            start = time.time()

In [7]:
# Get pruned model
net.load_state_dict(torch.load("pretrained_model.pt"))
for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        prune_by_percentage(layer, q=80.0)

# Training setup, do not change
batch_size=256
lr=0.002
reg=1e-4

print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=16)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.875, weight_decay=reg, nesterov=False)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [31]:
# Model finetuning
for epoch in range(20):
    print('\nEpoch: %d' % epoch)
    net.train()
    finetune_after_prune(net, trainloader, criterion, optimizer)
    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))

    if val_acc > best_acc:
        best_acc = val_acc
        print("Saving...")
        torch.save(net.state_dict(), "net_after_finetune.pt")


Epoch: 0
[Step=50]	Loss=0.9871	acc=0.6843	3646.7 examples/second
[Step=100]	Loss=0.8052	acc=0.7383	6326.0 examples/second
[Step=150]	Loss=0.7221	acc=0.7631	6432.9 examples/second
Test Loss=0.5583, Test acc=0.8190
Saving...

Epoch: 1
[Step=50]	Loss=0.4642	acc=0.8431	4185.5 examples/second
[Step=100]	Loss=0.4560	acc=0.8469	5489.9 examples/second
[Step=150]	Loss=0.4443	acc=0.8509	5323.7 examples/second
Test Loss=0.4871, Test acc=0.8394
Saving...

Epoch: 2
[Step=50]	Loss=0.3950	acc=0.8648	3748.7 examples/second
[Step=100]	Loss=0.3916	acc=0.8664	5102.9 examples/second
[Step=150]	Loss=0.3893	acc=0.8671	4968.4 examples/second
Test Loss=0.4558, Test acc=0.8509
Saving...

Epoch: 3
[Step=50]	Loss=0.3767	acc=0.8706	3752.1 examples/second
[Step=100]	Loss=0.3678	acc=0.8732	5580.7 examples/second
[Step=150]	Loss=0.3654	acc=0.8742	5691.2 examples/second
Test Loss=0.4352, Test acc=0.8580
Saving...

Epoch: 4
[Step=50]	Loss=0.3380	acc=0.8827	3818.9 examples/second
[Step=100]	Loss=0.3417	acc=0.8839	5205

In [11]:
# Check sparsity of the finetuned model, make sure it's not changed
net.load_state_dict(torch.load("net_after_finetune.pt"))

for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # Your code here:
        # Convert the weight of "layer" to numpy array
        np_weight = layer.weight.data.cpu().numpy()
        # Count number of zeros
        zeros = np.sum(np_weight == 0)
        # Count number of parameters
        total = np.prod(np_weight.shape)
        # Print sparsity
        print('Sparsity of '+name+': '+str(zeros/total))
        
test(net)

Sparsity of head_conv.0.conv: 0.7986111111111112
Sparsity of body_op.0.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.0.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.1.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.1.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.2.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.2.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.3.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.3.conv2.0.conv: 0.8000217013888888
Sparsity of body_op.4.conv1.0.conv: 0.8000217013888888
Sparsity of body_op.4.conv2.0.conv: 0.8000217013888888
Sparsity of body_op.5.conv1.0.conv: 0.8000217013888888
Sparsity of body_op.5.conv2.0.conv: 0.8000217013888888
Sparsity of body_op.6.conv1.0.conv: 0.7999674479166666
Sparsity of body_op.6.conv2.0.conv: 0.7999945746527778
Sparsity of body_op.7.conv1.0.conv: 0.7999945746527778
Sparsity of body_op.7.conv2.0.conv: 0.7999945746527778
Sparsity of body_op.8.conv1.0.conv: 0.7999945746527778
Sparsity of body

### Lab2 (d) Iterative pruning

In [13]:
net.load_state_dict(torch.load("pretrained_model.pt"))
best_acc = 0.
for epoch in range(20):
    print('\nEpoch: %d' % epoch)
    
    net.train()
    if epoch<10:
        for name,layer in net.named_modules():
            if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
                # Increase model sparsity
                # prune (8 × e)% before epoch e
                q = 8*(epoch+1)
                prune_by_percentage(layer, q=q)
    if epoch<9:
        finetune_after_prune(net, trainloader, criterion, optimizer,prune=False)
    else:
        finetune_after_prune(net, trainloader, criterion, optimizer)
    
    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))
    
    if epoch>=10:
        if val_acc > best_acc:
            best_acc = val_acc
            print("Saving...")
            torch.save(net.state_dict(), "net_after_iterative_prune.pt")


Epoch: 0
[Step=50]	Loss=0.0509	acc=0.9834	2795.6 examples/second
[Step=100]	Loss=0.0479	acc=0.9843	5643.8 examples/second
[Step=150]	Loss=0.0481	acc=0.9844	5255.0 examples/second
Test Loss=0.3249, Test acc=0.9155

Epoch: 1
[Step=50]	Loss=0.0547	acc=0.9819	3549.5 examples/second
[Step=100]	Loss=0.0524	acc=0.9831	4741.5 examples/second
[Step=150]	Loss=0.0520	acc=0.9834	5614.4 examples/second
Test Loss=0.3258, Test acc=0.9146

Epoch: 2
[Step=50]	Loss=0.0519	acc=0.9828	3527.0 examples/second
[Step=100]	Loss=0.0533	acc=0.9822	5016.2 examples/second
[Step=150]	Loss=0.0538	acc=0.9821	5491.6 examples/second
Test Loss=0.3299, Test acc=0.9121

Epoch: 3
[Step=50]	Loss=0.0575	acc=0.9802	3295.4 examples/second
[Step=100]	Loss=0.0570	acc=0.9809	5032.3 examples/second
[Step=150]	Loss=0.0561	acc=0.9810	5089.4 examples/second
Test Loss=0.3320, Test acc=0.9116

Epoch: 4
[Step=50]	Loss=0.0690	acc=0.9765	3659.1 examples/second
[Step=100]	Loss=0.0706	acc=0.9757	4194.2 examples/second
[Step=150]	Loss=0.067

In [14]:
# Check sparsity of the final model, make sure it's 80%
net.load_state_dict(torch.load("net_after_iterative_prune.pt"))

for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # Your code here: can copy from previous question
        # Convert the weight of "layer" to numpy array
        np_weight = layer.weight.data.cpu().numpy()
        # Count number of zeros
        zeros = np.sum(np_weight == 0)
        # Count number of parameters
        total = np.prod(np_weight.shape)
        # Print sparsity
        print('Sparsity of '+name+': '+str(zeros/total))        
        
test(net)

Sparsity of head_conv.0.conv: 0.7986111111111112
Sparsity of body_op.0.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.0.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.1.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.1.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.2.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.2.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.3.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.3.conv2.0.conv: 0.8000217013888888
Sparsity of body_op.4.conv1.0.conv: 0.8000217013888888
Sparsity of body_op.4.conv2.0.conv: 0.8000217013888888
Sparsity of body_op.5.conv1.0.conv: 0.8000217013888888
Sparsity of body_op.5.conv2.0.conv: 0.8000217013888888
Sparsity of body_op.6.conv1.0.conv: 0.7999674479166666
Sparsity of body_op.6.conv2.0.conv: 0.7999945746527778
Sparsity of body_op.7.conv1.0.conv: 0.7999945746527778
Sparsity of body_op.7.conv2.0.conv: 0.7999945746527778
Sparsity of body_op.8.conv1.0.conv: 0.7999945746527778
Sparsity of body

### Lab2 (e) Global iterative pruning

In [8]:
def global_prune_by_percentage(net, q=80.0):
    """
    Pruning the weight paramters by threshold.
    :param q: pruning percentile. 'q' percent of the least 
    significant weight parameters will be pruned.
    """
    # A list to gather all the weights
    flattened_weights = []
    # Find global pruning threshold
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Convert weight to numpy
            np_weight = layer.weight.data.cpu().numpy()
            # Flatten the weight and append to flattened_weights
            flattened_weights.append(np_weight.flatten())
    # Concate all weights into a np array
    flattened_weights = np.concatenate(flattened_weights)
    # Find global pruning threshold
    thres = np.percentile(np.abs(flattened_weights), q)
    
    # Apply pruning threshold to all layers
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Convert weight to numpy
            np_weight = layer.weight.data.cpu().numpy()
            # Generate a binary mask same shape as weight to decide which element to prune
            mask = np.abs(np_weight) > thres
            # Convert mask to torch tensor and put on GPU
            mask = torch.from_numpy(mask).to(device)
            # Multiply the weight by mask to perform pruning
            layer.weight.data = layer.weight.data * mask

In [9]:
net.load_state_dict(torch.load("pretrained_model.pt"))
best_acc = 0.
for epoch in range(20):
    print('\nEpoch: %d' % epoch)
    q=(epoch+1)*8
    
    net.train()
    # Increase model sparsity
    if epoch<10:
        global_prune_by_percentage(net, q=q)
    if epoch<9:
        finetune_after_prune(net, trainloader, criterion, optimizer,prune=False)
    else:
        finetune_after_prune(net, trainloader, criterion, optimizer)
    
    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))
    
    if epoch>=10:
        if val_acc > best_acc:
            best_acc = val_acc
            print("Saving...")
            torch.save(net.state_dict(), "net_after_global_iterative_prune.pt")


Epoch: 0
[Step=50]	Loss=0.0492	acc=0.9838	3212.4 examples/second
[Step=100]	Loss=0.0491	acc=0.9845	4469.8 examples/second
[Step=150]	Loss=0.0489	acc=0.9847	4811.2 examples/second
Test Loss=0.3256, Test acc=0.9141

Epoch: 1
[Step=50]	Loss=0.0512	acc=0.9840	3716.3 examples/second
[Step=100]	Loss=0.0504	acc=0.9842	5080.0 examples/second
[Step=150]	Loss=0.0507	acc=0.9839	4835.3 examples/second
Test Loss=0.3250, Test acc=0.9159

Epoch: 2
[Step=50]	Loss=0.0477	acc=0.9851	3794.4 examples/second
[Step=100]	Loss=0.0484	acc=0.9851	4769.4 examples/second
[Step=150]	Loss=0.0491	acc=0.9850	5161.2 examples/second
Test Loss=0.3268, Test acc=0.9147

Epoch: 3
[Step=50]	Loss=0.0523	acc=0.9831	3760.4 examples/second
[Step=100]	Loss=0.0527	acc=0.9825	4539.5 examples/second
[Step=150]	Loss=0.0534	acc=0.9821	4801.4 examples/second
Test Loss=0.3277, Test acc=0.9134

Epoch: 4
[Step=50]	Loss=0.0570	acc=0.9816	4244.8 examples/second
[Step=100]	Loss=0.0562	acc=0.9817	4502.7 examples/second
[Step=150]	Loss=0.057

In [10]:
net.load_state_dict(torch.load("net_after_global_iterative_prune.pt"))

zeros_sum = 0
total_sum = 0
for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # Your code here:
        # Convert the weight of "layer" to numpy array
        np_weight = layer.weight.data.cpu().numpy()
        # Count number of zeros
        zeros = np.sum(np_weight == 0)
        # Count number of parameters
        total = np.prod(np_weight.shape)
        zeros_sum+=zeros
        total_sum+=total
        print('Sparsity of '+name+': '+str(zeros/total))
print('Total sparsity of: '+str(zeros_sum/total_sum))
test(net)

Sparsity of head_conv.0.conv: 0.3101851851851852
Sparsity of body_op.0.conv1.0.conv: 0.6571180555555556
Sparsity of body_op.0.conv2.0.conv: 0.6388888888888888
Sparsity of body_op.1.conv1.0.conv: 0.6267361111111112
Sparsity of body_op.1.conv2.0.conv: 0.6480034722222222
Sparsity of body_op.2.conv1.0.conv: 0.6302083333333334
Sparsity of body_op.2.conv2.0.conv: 0.6684027777777778
Sparsity of body_op.3.conv1.0.conv: 0.6247829861111112
Sparsity of body_op.3.conv2.0.conv: 0.6882595486111112
Sparsity of body_op.4.conv1.0.conv: 0.7259114583333334
Sparsity of body_op.4.conv2.0.conv: 0.7823350694444444
Sparsity of body_op.5.conv1.0.conv: 0.7249348958333334
Sparsity of body_op.5.conv2.0.conv: 0.8130425347222222
Sparsity of body_op.6.conv1.0.conv: 0.7326388888888888
Sparsity of body_op.6.conv2.0.conv: 0.764404296875
Sparsity of body_op.7.conv1.0.conv: 0.7772081163194444
Sparsity of body_op.7.conv2.0.conv: 0.8260633680555556
Sparsity of body_op.8.conv1.0.conv: 0.8526746961805556
Sparsity of body_op.

### Lab 3 (b) and (c): Fixed-point quantization

In [4]:
# Define quantized model and load weight
Nbits = None #Change this value to finish (b) and (c)

net = ResNetCIFAR(num_layers=20, Nbits=Nbits)
net = net.to(device)
net.load_state_dict(torch.load("pretrained_model.pt"))
test(net)

Files already downloaded and verified
Test Loss=0.3231, Test accuracy=0.9151


In [2]:
nbits_lst = [6, 5, 4, 3, 2]

for nbit in nbits_lst:
    Nbits = nbit
    net = ResNetCIFAR(num_layers=20, Nbits=Nbits)
    net = net.to(device)
    net.load_state_dict(torch.load("pretrained_model.pt"))
    test(net)

Files already downloaded and verified
Test Loss=0.3364, Test accuracy=0.9145
Files already downloaded and verified
Test Loss=0.3390, Test accuracy=0.9112
Files already downloaded and verified
Test Loss=0.3861, Test accuracy=0.8972
Files already downloaded and verified
Test Loss=0.9874, Test accuracy=0.7662
Files already downloaded and verified
Test Loss=9.5441, Test accuracy=0.0899


In [ ]:
nbit_lst = [4, 3, 2]

for nbit in nbit_lst:
    print("Nbits:", nbit)
    net = ResNetCIFAR(num_layers=20, Nbits=nbit)
    net = net.to(device)
    net.load_state_dict(torch.load("pretrained_model.pt"))
    # Quantized model finetuning
    finetune(net, epochs=20, batch_size=256, lr=0.002, reg=1e-4)   

    # Load the model with best accuracy
    net.load_state_dict(torch.load("quantized_net_after_finetune.pt"))
    test(net)

Nbits: 4
==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0
[Step=50]	Loss=0.0702	acc=0.9745	2546.7 examples/second
[Step=100]	Loss=0.0658	acc=0.9770	3819.3 examples/second
[Step=150]	Loss=0.0663	acc=0.9771	4591.5 examples/second
Test Loss=0.3344, Test acc=0.9071
Saving...

Epoch: 1
[Step=200]	Loss=0.0464	acc=0.9834	1775.8 examples/second
[Step=250]	Loss=0.0664	acc=0.9777	4936.4 examples/second
[Step=300]	Loss=0.0643	acc=0.9781	5021.0 examples/second
[Step=350]	Loss=0.0637	acc=0.9786	5016.0 examples/second
Test Loss=0.3363, Test acc=0.9105
Saving...

Epoch: 2
[Step=400]	Loss=0.0461	acc=0.9878	1731.2 examples/second
[Step=450]	Loss=0.0594	acc=0.9802	4163.2 examples/second
[Step=500]	Loss=0.0614	acc=0.9789	4161.8 examples/second
[Step=550]	Loss=0.0616	acc=0.9788	4364.8 examples/second
Test Loss=0.3328, Test acc=0.9093

Epoch: 3
[Step=600]	Loss=0.0543	acc=0.9814	1972.9 examples/second
[Step=650]	Loss=0.0539	acc=0.9820	4482.4 examples

In [2]:
print("Nbits:", 2)
net = ResNetCIFAR(num_layers=20, Nbits=2)
net = net.to(device)
net.load_state_dict(torch.load("pretrained_model.pt"))
# Quantized model finetuning
finetune(net, epochs=20, batch_size=256, lr=0.002, reg=1e-4)   

# Load the model with best accuracy
net.load_state_dict(torch.load("quantized_net_after_finetune.pt"))
test(net)

Nbits: 2
==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0
[Step=50]	Loss=1.3528	acc=0.6161	2556.7 examples/second
[Step=100]	Loss=1.0749	acc=0.6821	4770.0 examples/second
[Step=150]	Loss=0.9471	acc=0.7139	4799.2 examples/second
Test Loss=0.7775, Test acc=0.7664
Saving...

Epoch: 1
[Step=200]	Loss=0.5379	acc=0.8076	1669.8 examples/second
[Step=250]	Loss=0.5697	acc=0.8090	4771.2 examples/second
[Step=300]	Loss=0.5573	acc=0.8119	4488.7 examples/second
[Step=350]	Loss=0.5434	acc=0.8162	4716.9 examples/second
Test Loss=0.6802, Test acc=0.7972
Saving...

Epoch: 2
[Step=400]	Loss=0.4659	acc=0.8442	1814.0 examples/second
[Step=450]	Loss=0.4764	acc=0.8353	4435.6 examples/second
[Step=500]	Loss=0.4680	acc=0.8383	4592.0 examples/second
[Step=550]	Loss=0.4647	acc=0.8396	4742.3 examples/second
Test Loss=0.6145, Test acc=0.8114
Saving...

Epoch: 3
[Step=600]	Loss=0.4342	acc=0.8532	1774.1 examples/second
[Step=650]	Loss=0.4387	acc=0.8474	4420.

### Lab3 (d) Quantize pruned model

In [ ]:
Nbits = 4 
net = ResNetCIFAR(num_layers=20, Nbits=Nbits)
net = net.to(device)
net.load_state_dict(torch.load("net_after_global_iterative_prune.pt"))
test(net)
finetune(net, epochs=20, batch_size=256, lr=0.002, reg=1e-4)

# Load the model with best accuracy
net.load_state_dict(torch.load("quantized_net_after_finetune.pt"))
test(net)

In [13]:
# Define quantized model and load weight
Nbits = 3 #Change this value to finish (d)

net = ResNetCIFAR(num_layers=20, Nbits=Nbits)
net = net.to(device)
net.load_state_dict(torch.load("net_after_global_iterative_prune.pt"))
test(net)

Files already downloaded and verified
Test Loss=4.2843, Test accuracy=0.0924


In [ ]:
# Quantized model finetuning
finetune(net, epochs=20, batch_size=256, lr=0.002, reg=1e-4)

# Load the model with best accuracy
net.load_state_dict(torch.load("quantized_net_after_finetune.pt"))
test(net)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0
[Step=50]	Loss=2.1449	acc=0.3780	3386.1 examples/second
[Step=100]	Loss=1.7209	acc=0.4602	4246.5 examples/second
[Step=150]	Loss=1.5050	acc=0.5140	4636.2 examples/second
Test Loss=0.8544, Test acc=0.7007
Saving...

Epoch: 1
[Step=200]	Loss=0.8195	acc=0.6914	1666.9 examples/second
[Step=250]	Loss=0.8017	acc=0.7169	4636.7 examples/second
[Step=300]	Loss=0.7591	acc=0.7332	3975.3 examples/second
[Step=350]	Loss=0.7270	acc=0.7440	4027.8 examples/second
Test Loss=0.6286, Test acc=0.7846
Saving...

Epoch: 2
[Step=400]	Loss=0.6057	acc=0.7905	1492.9 examples/second
[Step=450]	Loss=0.5814	acc=0.8015	4015.6 examples/second
[Step=500]	Loss=0.5707	acc=0.8045	3862.4 examples/second
[Step=550]	Loss=0.5620	acc=0.8079	4341.6 examples/second
Test Loss=0.5724, Test acc=0.8055
Saving...

Epoch: 3
[Step=600]	Loss=0.5153	acc=0.8301	1607.5 examples/second
[Step=650]	Loss=0.5012	acc=0.8325	4037.1 example

In [ ]:
Nbits = 2
net = ResNetCIFAR(num_layers=20, Nbits=Nbits)
net = net.to(device)
net.load_state_dict(torch.load("net_after_global_iterative_prune.pt"))
test(net)
finetune(net, epochs=20, batch_size=256, lr=0.002, reg=1e-4)

# Load the model with best accuracy
net.load_state_dict(torch.load("quantized_net_after_finetune.pt"))
test(net)

### Lab3 (e) Prune quantized model

In [ ]:
net = ResNetCIFAR(num_layers=20, Nbits=4)
net = net.to(device)

net.load_state_dict(torch.load("pretrained_model.pt"))
best_acc = 0.
for epoch in range(20):
    print('\nEpoch: %d' % epoch)
    q=(epoch+1)*8
    
    net.train()
    # Increase model sparsity
    if epoch<10:
        global_prune_by_percentage(net, q=q)
    if epoch<9:
        finetune_after_prune(net, trainloader, criterion, optimizer,prune=False)
    else:
        finetune_after_prune(net, trainloader, criterion, optimizer)
    
    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))
    
    if epoch>=10:
        if val_acc > best_acc:
            best_acc = val_acc
            print("Saving...")
            torch.save(net.state_dict(), "quantized_net_after_global_iterative_prune.pt")

In [ ]:
net.load_state_dict(torch.load("quantized_net_after_global_iterative_prune.pt"))

zeros_sum = 0
total_sum = 0
for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        np_weight = layer.weight.data.cpu().numpy()
        zeros = (np_weight==0).sum()
        total = np_weight.flatten().shape[0]
        zeros_sum+=zeros
        total_sum+=total
        print('Sparsity of '+name+': '+str(zeros/total))
print('Total sparsity of: '+str(zeros_sum/total_sum))
test(net)